# Import used packages

In [1]:
import pandas as pd
import folium
import branca.colormap as cm
from h3 import h3
from shapely.geometry import Polygon

# Import data and convert to geopandas df

In [2]:
trips = pd.read_csv('../data/processed/bremen.csv')

In [3]:
trips.head(2)

,bike,identification,start_time,end_time,weekend,duration_sec,start_lng,start_lat,end_lng,end_lat,...,humidity_2m,dew_point_2m,max_at_2m,mean_speed_h/s,direction_degree_x,max_m/s,min_mean_m/s,max_mean_m/s,direction_degree_y,min
0,20507,14776184,2019-04-21 17:34:00,2019-04-21 17:47:00,1,780.0,8.884911,53.078923,8.899906,53.078711,...,24.8,-0.1,21.1,1.8,120.0,2.1,1.0,2.2,110.0,0.0
1,20507,10278649,2019-04-23 17:55:00,2019-04-23 20:13:00,0,8280.0,8.884911,53.078923,8.884911,53.078923,...,43.2,5.9,18.7,6.7,110.0,11.7,3.6,7.3,110.0,0.0


## Add h3 hexagon (resolution 9) addresses to trips data frame

In [4]:
resolution = 9

sHex = []
eHex = []

for index, row in trips.iterrows():
    sHex.append(h3.geo_to_h3(row['start_lat'], row['start_lng'], resolution))
    eHex.append(h3.geo_to_h3(row['end_lat'], row['end_lng'], resolution))

In [5]:
trips['sHex'] = sHex
trips['eHex'] = eHex

In [6]:
trips.head(2)

,bike,identification,start_time,end_time,weekend,duration_sec,start_lng,start_lat,end_lng,end_lat,...,max_at_2m,mean_speed_h/s,direction_degree_x,max_m/s,min_mean_m/s,max_mean_m/s,direction_degree_y,min,sHex,eHex
0,20507,14776184,2019-04-21 17:34:00,2019-04-21 17:47:00,1,780.0,8.884911,53.078923,8.899906,53.078711,...,21.1,1.8,120.0,2.1,1.0,2.2,110.0,0.0,891f154a56fffff,891f154a187ffff
1,20507,10278649,2019-04-23 17:55:00,2019-04-23 20:13:00,0,8280.0,8.884911,53.078923,8.884911,53.078923,...,18.7,6.7,110.0,11.7,3.6,7.3,110.0,0.0,891f154a56fffff,891f154a56fffff


In [7]:
trips['count'] = 1 # Add count column to count number of trips per hexagon

## Visualize start locations

In [8]:
starts = trips.groupby(['sHex']).agg({'count':'sum'}) # Aggregate all starts related to the corresponding hexagon

In [9]:
starts.reset_index(inplace=True)

There are many hexagons with only one trip in 2019, so we visualize only hexagons with at least 50 trips in 2019.

In [10]:
starts.sort_values(by='count')

,sHex,count
534,891f154acb3ffff,1
531,891f154ac8fffff,1
530,891f154ac7bffff,1
527,891f154ac63ffff,1
525,891f154ac47ffff,1
...,...,...
910,891f15599cfffff,2078
880,891f1559947ffff,2118
906,891f15599bbffff,2990
884,891f1559957ffff,3270


## Functions for visualization

In [11]:
# Function that takes a list of and latitude and longitude coordinates reverses them and returns a list.
def reverse_lat_lon(hex_coords):
    geom_hex = []

    for lat_lon in hex_coords:
        geom_hex.append([lat_lon[1], lat_lon[0]])

    return geom_hex

# Function that draws a polygon onto a folium map.
def add_poly_to_map(polygon, m, color, fillColor, weight, colormap):
    
    style_function = lambda x: {"weight":0, 
                                'color':'black',
                                'fillColor':colormap(weight), 
                                'fillOpacity':0.4}
    
    popup = folium.Popup('Number of trips in 2019: ' + str(weight))
    
    gj = folium.GeoJson(polygon, style_function=style_function)
    
    gj.add_child(popup)
    
    gj.add_to(m)

# Function that draws one or multiple h3 hexagon onto a folium map.
def add_hex_to_map(hex_address, m, color, fillColor, weight, colormap):
    if isinstance(hex_address, str):
        hex_poly = Polygon(reverse_lat_lon(h3.h3_to_geo_boundary(h3_address=hex_address)))

        add_poly_to_map(hex_poly, m, color, fillColor, weight, colormap)

    if isinstance(hex_address, list):

        for address in hex_address:
            hex_poly = Polygon(reverse_lat_lon(h3.h3_to_geo_boundary(h3_address=address)))

            add_poly_to_map(hex_poly, m, color, fillColor, weight, colormap)

In [12]:
# Create map
m = folium.Map(location=[53.0792962,8.8016937], zoom_start=12)

# Add style elements to map
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(m)
colormap = cm.LinearColormap(colors=['blue','red'], index=[50,3800],vmin=0,vmax=3800) # Create colormap
colormap.caption = "number of starts in 2019"
colormap.add_to(m) # Add colormap to map

# Add hexagons with more than 49 to map
for index, row in starts.sort_values(by='count')[:len(starts)-1][starts['count'] > 49].iterrows():
    weight = row['count']
    add_hex_to_map(row['sHex'], m, 'royalblue', 'royalblue', weight, colormap)

# Add hexagon with most trips to the map - seperately for nicer visualization
add_hex_to_map('891f1559917ffff', m, 'royalblue', 'royalblue', 9799, colormap)

# Save map to reports
m.save('../reports/figures/start_locations_hexmap_2019.html')

# Show map
m

/Users/timockenga/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # This is added back by InteractiveShellApp.init_path()


## Visualize end locations

In [13]:
ends = trips.groupby(['eHex']).agg({'count':'sum'}) # Aggregate all ends related to the corresponding hexagon

In [14]:
ends.reset_index(inplace=True)

There are many hexagons with only one trip in 2019, so we visualize only hexagons with at least 50 trips in 2019.

In [15]:
ends.sort_values(by='count')

,eHex,count
1101,891f155b597ffff,1
175,891f15485b7ffff,1
658,891f15581c7ffff,1
657,891f1558177ffff,1
178,891f1548693ffff,1
...,...,...
933,891f15599cfffff,2010
903,891f1559947ffff,2076
929,891f15599bbffff,2926
907,891f1559957ffff,3205


In [16]:
# Create map
m = folium.Map(location=[53.0792962,8.8016937], zoom_start=12) # Create map

# Add style elements to map
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(m)
colormap = cm.LinearColormap(colors=['blue','red'], index=[50,3800],vmin=0,vmax=3800) # Create colormap
colormap.caption = "number of starts in 2019"
colormap.add_to(m) # Add colormap to map

# Add hexagons with more than 49 to map
for index, row in ends.sort_values(by='count')[:len(ends)-1][ends['count'] >= 50].iterrows():
    weight = row['count']
    add_hex_to_map(row['eHex'], m, 'royalblue', 'royalblue', weight, colormap)

# Add hexagon with most trips to the map - seperately for nicer visualization
add_hex_to_map('891f1559917ffff', m, 'royalblue', 'royalblue', 9406, colormap)

# Save map to reports
m.save('../reports/figures/end_locations_hexmap_2019.html')

# Show map
m

/Users/timockenga/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # This is added back by InteractiveShellApp.init_path()


As one can see start and end locations are almost the same regarding the number of trips related to them. In genereal, there are some more end locations (1155 - 1134 = 21) than start locations (for hexagons with resolution of 9) that have at least 50 related trips in 2019.